# Dashboard v1.0
3 dic 2023


### David Escudero Garcia

In [2]:
import pandas as pd
import numpy as np
import pulp as plp
import geopandas as gpd
import os
import requests
from shapely.geometry import Point
from shapely.ops import cascaded_union
import logging
import psycopg2
import time
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
from sqlalchemy import create_engine, text
import geopandas as gpd
import plotly.express as px
import dash_core_components as dcc
import sqlalchemy as sa


## Functions

In [3]:
# params
logging.basicConfig(level=logging.DEBUG)
FLASK_ENV = os.environ.get('FLASK_ENV')

# database connection
os.environ['POSTGRES_DB'] = 'fitpass'
os.environ['POSTGRES_USER'] = 'postgres'
os.environ['POSTGRES_HOST'] = 'localhost'
os.environ['POSTGRES_PASSWORD'] = 'skalas-puts-me-an-aplus-in-this-class'


1. `logging.basicConfig(level=logging.DEBUG)`: Esta línea configura el sistema de registro (logging) de Python para que muestre mensajes de depuración (DEBUG). El nivel DEBUG es uno de los niveles de gravedad más bajos para los mensajes de registro, lo que significa que se mostrarán mensajes detallados sobre el comportamiento de la aplicación, útiles durante el desarrollo o la depuración.

2. `FLASK_ENV = os.environ.get('FLASK_ENV')`: Aquí, se está recuperando una variable de entorno llamada 'FLASK_ENV' y se está asignando su valor a la variable `FLASK_ENV`. Esto se utiliza comúnmente para establecer el entorno en el que se ejecuta una aplicación Flask (por ejemplo, desarrollo, producción, etc.). Si la variable de entorno no está establecida, `FLASK_ENV` será `None`.

3. Las siguientes líneas establecen variables de entorno para la conexión a una base de datos PostgreSQL:
   - `os.environ['POSTGRES_DB'] = 'fitpass'`: Establece el nombre de la base de datos a la que se conectará la aplicación.
   - `os.environ['POSTGRES_USER'] = 'postgres'`: Establece el nombre de usuario para la conexión a la base de datos.
   - `os.environ['POSTGRES_HOST'] = 'localhost'`: Define la ubicación del servidor de base de datos (en este caso, el servidor local).
   - `os.environ['POSTGRES_PASSWORD'] = 'skalas-puts-me-an-aplus-in-this-class'`: Establece la contraseña para la conexión a la base de datos.

**Estas líneas son típicas en scripts de configuración donde se prepara el entorno de ejecución para una aplicación, particularmente para definir parámetros de conexión a la base de datos y configurar el nivel de detalle de los registros de la aplicación.**

In [4]:
def log_debugg(text):
    if FLASK_ENV == 'development':
        logging.debug(text)

def get_db_conn():
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            log_debugg(f"Trying to connect to the PostgreSQL database... ({retries}/{max_retries})")
            host = os.environ.get('POSTGRES_HOST', 'localhost')
            user = os.environ.get('POSTGRES_USER', 'postgres')
            password = os.environ.get('POSTGRES_PASSWORD', '')
            database = os.environ.get('POSTGRES_DB', 'fitpass')
            conn = create_engine(f'postgresql://{user}:{password}@{host}/{database}')
            log_debugg("Connected to the PostgreSQL database.")
            return conn
        except psycopg2.OperationalError as e:
            log_debugg(f"Error: {e}")
            log_debugg(f"Waiting 10 seconds for PostgreSQL to be ready... ({retries}/{max_retries})")
            retries += 1
            time.sleep(10)

    log_debugg("Max retries reached. Unable to connect to the PostgreSQL database.")
    return None

1. **Función `log_debugg(text)`**:
   - Esta función es un envoltorio personalizado para la función de registro de depuración (`logging.debug`).
   - Comprueba si la variable de entorno `FLASK_ENV` está establecida en 'development'. Si es así, procede a registrar el mensaje de depuración proporcionado.
   - El propósito de esta comprobación es registrar mensajes detallados solo cuando la aplicación está en modo de desarrollo, lo que es útil para depurar sin abrumar la salida de registro en un entorno de producción.

2. **Función `get_db_conn()`**:
   - Esta función intenta establecer una conexión con una base de datos PostgreSQL.
   - Utiliza un bucle `while` para intentar conectarse hasta un máximo de tres veces (`max_retries = 3`).
   - Dentro del bucle, utiliza la función `log_debugg` para registrar el intento de conexión y cualquier error que pueda ocurrir.
   - Recupera los detalles de conexión (host, usuario, contraseña y nombre de la base de datos) de las variables de entorno. Si no están establecidas, utiliza valores predeterminados.
   - Intenta crear una conexión a la base de datos utilizando `create_engine` de SQLAlchemy (implicado por la sintaxis `create_engine`).
   - Si se produce un error de conexión (capturado como `psycopg2.OperationalError`), registra el error, espera 10 segundos y luego lo intenta de nuevo.
   - Si se alcanza el máximo de reintentos sin éxito, registra un mensaje de error y devuelve `None`, indicando que no se pudo establecer la conexión.

**Estas funciones son útiles en aplicaciones web, especialmente aquellas que utilizan Flask y PostgreSQL, proporcionando una manera de gestionar las conexiones a la base de datos con una estrategia de reintentos y registros útiles para la depuración.**

## Connection

In [5]:
log_debugg("reading data")
conn = get_db_conn()
query = "select * from cdmx_studios" 
df_fitpass_r = pd.read_sql_query(query, conn)
df_fitpass_r = df_fitpass_r.drop_duplicates(subset=['gym_id']) # drop duplicates
# conn.dispose() # close connection

El fragmento de código  realiza las siguientes acciones en el contexto de una aplicación web basada en Flask que interactúa con una base de datos PostgreSQL:

1. **Registro de la Acción de Lectura de Datos**:
   - `log_debugg("reading data")`: Esta línea utiliza la función `log_debugg` definida anteriormente para registrar el inicio de la acción de lectura de datos. Si `FLASK_ENV` está configurado como 'development', mostrará un mensaje de depuración.

2. **Establecimiento de Conexión a la Base de Datos**:
   - `conn = get_db_conn()`: Aquí se llama a la función `get_db_conn`, también definida anteriormente, para obtener una conexión a la base de datos PostgreSQL. Si no puede establecer la conexión después de los intentos de reintento, `conn` será `None`.

3. **Ejecución de la Consulta SQL y Carga en un DataFrame de pandas**:
   - `df_fitpass_r = pd.read_sql_query(query, conn)`: Esta línea ejecuta una consulta SQL (almacenada en la variable `query`, que selecciona todas las filas de la tabla `cdmx_studios`) y carga los resultados en un DataFrame de pandas llamado `df_fitpass_r`. pandas utiliza la conexión a la base de datos proporcionada (`conn`) para ejecutar la consulta.

4. **Eliminación de Filas Duplicadas**:
   - `df_fitpass_r = df_fitpass_r.drop_duplicates(subset=['gym_id'])`: Esta línea elimina las filas duplicadas en el DataFrame `df_fitpass_r` basándose en el `gym_id`. Esto significa que si hay múltiples filas con el mismo `gym_id`, solo se mantendrá la primera y las demás serán descartadas.

5. **Cierre de la Conexión a la Base de Datos**:
   - `# conn.dispose()`: Esta línea está comentada, pero si se descomentara, cerraría la conexión a la base de datos. `dispose` es un método que se utiliza generalmente para cerrar conexiones de SQLAlchemy. Sin embargo, dependiendo de cómo esté configurado el manejo de conexiones en la aplicación, podría no ser necesario llamar a `dispose` explícitamente, ya que muchas implementaciones manejan el cierre de la conexión automáticamente.


In [6]:
df_fitpass_r

,gym_id,gym_name,pro_status,virtual_status,class_minutes,latitude,longitude,barre,box,crossfit,...,gym,hiit,mma,pilates,pool,running,sports,virtual_class,wellness,yoga
0,3913,21159 Fit Movement,0,0,45,19.409432,-99.162129,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,4356,321 Move!,0,0,None,19.359657,-99.202687,1.0,1.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,5576,3 Are Legend,0,0,None,19.539159,-99.182145,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3735,40 Grados Hot Yoga,0,0,None,19.552389,-99.270923,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5694,60 Mind Fitness,0,0,None,19.401965,-99.155869,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861,17,Zonadanza: Ballet Studio & Art,0,0,None,19.390768,-99.291066,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
862,1623,Zuda Lilas,1,0,None,19.38801,-99.247764,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
863,3976,Zuda Prado Norte,1,0,None,19.427231,-99.211101,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
864,3101,Zuda Virtual,0,1,None,19.405721,-99.252566,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
df_fitpass_r

## Request API

In [7]:
# sample
sample_request = {
    "name": "roman",
    "location": {
        "latitude": 19.388900864307445,
        "longitude": -99.18265186842596
    },
    "distance_sensitivity": "medium",
    "preferences": {
        "love_activities": ["barre", "yoga", "cycling", "pilates", "gym"],
        "hate_activities": ["crossfit", "functional"]
    },
    "is_pro": 1,
    "max_allowed_classes_per_class": 4,
    "num_classes_per_month": 23
}

Este fragmento de código define un objeto `sample_request` en Python, que representa una muestra de solicitud para una API. La estructura y los datos del objeto sugieren que está destinado para un servicio que ofrece recomendaciones o reservas para clases de fitness o actividades relacionadas. Aquí está el análisis detallado de cada parte del objeto:

1. **Información Básica del Usuario**:
   - `"name": "roman"`: El nombre del usuario es "roman".
   - `"is_pro": 1`: Indica si el usuario es un profesional o tiene algún status avanzado. El valor `1` probablemente indica un estado 'verdadero' o 'sí'.

2. **Localización**:
   - `"location": { "latitude": 19.388900864307445, "longitude": -99.18265186842596 }`: Define la ubicación geográfica del usuario mediante latitud y longitud. Los valores proporcionados corresponden a coordenadas específicas, que podrían ser utilizadas para localizar estudios o clases de fitness cerca del usuario.

3. **Sensibilidad a la Distancia**:
   - `"distance_sensitivity": "medium"`: Esto podría referirse a cuán lejos está dispuesto a viajar el usuario para asistir a una clase. En este caso, la sensibilidad a la distancia es "media", lo que sugiere una preferencia por lugares ni demasiado cerca ni demasiado lejos.

4. **Preferencias de Actividades**:
   - `"preferences": { "love_activities": ["barre", "yoga", "cycling", "pilates", "gym"], "hate_activities": ["crossfit", "functional"] }`: Esta sección detalla las preferencias del usuario en cuanto a tipos de actividades físicas. "love_activities" son las que el usuario disfruta o prefiere, mientras que "hate_activities" son aquellas que el usuario no prefiere o desea evitar.

5. **Restricciones de Clases**:
   - `"max_allowed_classes_per_class": 4`: Podría indicar el máximo número de veces que el usuario puede asistir a la misma clase.
   - `"num_classes_per_month": 23`: Indica el número total de clases a las que el usuario planea asistir en un mes.

Este objeto `sample_request` podría ser utilizado como cuerpo de una solicitud HTTP POST en una API web, donde un servidor procesaría estos datos para ofrecer recomendaciones personalizadas de clases o servicios relacionados con el fitness basados en la ubicación, preferencias y restricciones del usuario.

In [8]:
# generate post request to http://localhost:8080/predict
url = 'http://localhost:8080/predict'
r = requests.post(url, json=sample_request)
r.json()

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:8080


DEBUG:urllib3.connectionpool:http://localhost:8080 "POST /predict HTTP/1.1" 200 353


[{'gym_id': '5305', 'gym_times': 1},
 {'gym_id': '681', 'gym_times': 1},
 {'gym_id': '2376', 'gym_times': 1},
 {'gym_id': '1051', 'gym_times': 1},
 {'gym_id': '2621', 'gym_times': 1},
 {'gym_id': '2634', 'gym_times': 1},
 {'gym_id': '2646', 'gym_times': 1},
 {'gym_id': '5350', 'gym_times': 4},
 {'gym_id': '5410', 'gym_times': 4},
 {'gym_id': '1067', 'gym_times': 4},
 {'gym_id': '2628', 'gym_times': 4}]

El fragmento de código proporcionado muestra cómo hacer una solicitud POST a un servidor web local utilizando Python y la biblioteca `requests`. Aquí está el desglose de lo que hace cada línea:

1. **Definición de la URL**:
   - `url = 'http://localhost:8080/predict'`: Aquí, se define la URL a la que se va a hacer la solicitud POST. Esta URL apunta a un servidor local (`localhost`) en el puerto `8080` en el endpoint `/predict`.

2. **Realización de la Solicitud POST**:
   - `r = requests.post(url, json=sample_request)`: Se utiliza la función `post` de la biblioteca `requests` para realizar una solicitud POST a la URL especificada. 
   - El argumento `json=sample_request` indica que los datos enviados en la solicitud POST están en formato JSON y corresponden al objeto `sample_request` definido anteriormente.
   - La respuesta del servidor a esta solicitud POST se almacena en la variable `r`.

3. **Obtención y Retorno de la Respuesta JSON**:
   - `r.json()`: Esta línea convierte la respuesta del servidor (asumiendo que es un objeto JSON) a un diccionario Python para su fácil manipulación o acceso en el código. 
   - Si el servidor devuelve una respuesta en formato JSON, esta línea permite acceder a los datos de la respuesta.

Este código se usaría típicamente en una aplicación donde necesitas comunicarte con un servidor backend que procesa datos y devuelve una respuesta. Por ejemplo, en este caso, el servidor en `http://localhost:8080/predict` podría ser una API que procesa información de usuario para predicciones o recomendaciones (como en un sistema de recomendación de clases de fitness, basado en el objeto `sample_request` mencionado anteriormente).

In [9]:
df_final_product = pd.DataFrame(r.json())

La línea de código convierte la respuesta JSON de una solicitud HTTP POST en un DataFrame de pandas.

- `df_final_product = pd.DataFrame(r.json())`: Esta línea realiza las siguientes acciones:

  1. **Extraer JSON de la Respuesta**: `r.json()` convierte la respuesta JSON del servidor (almacenada en la variable `r`) en un diccionario de Python. La función `json()` es un método de la clase `Response` de la biblioteca `requests`, que se utiliza para decodificar la respuesta del servidor en formato JSON.

  2. **Crear DataFrame de pandas**: `pd.DataFrame()` toma el diccionario decodificado y lo convierte en un DataFrame de pandas. Pandas es una biblioteca en Python que proporciona estructuras de datos y herramientas para el análisis de datos. Un DataFrame es una estructura de datos bidimensional, similar a una tabla, que puede contener diferentes tipos de datos y es muy útil para el análisis de datos.

Esta línea de código es útil cuando se trabaja con APIs que devuelven datos en formato JSON y se quiere analizar o manipular esos datos utilizando pandas en Python. Por ejemplo, si la API devuelve datos relacionados con las recomendaciones de clases de fitness (como se sugirió en tu ejemplo anterior del objeto `sample_request`), este DataFrame permitiría un análisis más detallado o manipulaciones adicionales de esos datos, como filtrado, agrupación, visualización, etc.

In [10]:
df_final_product

,gym_id,gym_times
0,5305,1
1,681,1
2,2376,1
3,1051,1
4,2621,1
5,2634,1
6,2646,1
7,5350,4
8,5410,4
9,1067,4


In [11]:
df_work = df_final_product.merge(df_fitpass_r, on='gym_id', how='inner')
df_work

,gym_id,gym_times,gym_name,pro_status,virtual_status,class_minutes,latitude,longitude,barre,box,...,gym,hiit,mma,pilates,pool,running,sports,virtual_class,wellness,yoga
0,5305,1,Beatness Cycling Origami,1,0,None,19.368401,-99.18044,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,681,1,Body Fitness Pilates Del Valle,1,0,None,19.393445,-99.166104,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2376,1,Dragonfly Pole Del Valle,1,0,None,19.397872,-99.159608,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,1051,1,El T3mplo Altavista Outdoors,1,0,None,19.349584,-99.196501,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
4,2621,1,Sports World Félix Cuevas,1,0,None,19.372945,-99.172982,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2634,1,Sports World Manacar,1,0,None,19.368492,-99.181021,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,2646,1,Sports World Xola,1,0,None,19.397099,-99.166053,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,5350,4,Equilibre et Relax,1,0,None,19.375681,-99.177941,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
8,5410,4,Fever Club Del Valle Carracci,1,0,None,19.374673,-99.178886,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
9,1067,4,Sport City Eureka,1,0,None,19.386204,-99.191041,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
df_work.columns

Index(['gym_id', 'gym_times', 'gym_name', 'pro_status', 'virtual_status',
       'class_minutes', 'latitude', 'longitude', 'barre', 'box', 'crossfit',
       'cycling', 'dance', 'ems', 'functional', 'gym', 'hiit', 'mma',
       'pilates', 'pool', 'running', 'sports', 'virtual_class', 'wellness',
       'yoga'],
      dtype='object')

El código  realiza una operación de fusión (merge) entre dos DataFrames en pandas: `df_final_product` y `df_fitpass_r`.

- `df_work = df_final_product.merge(df_fitpass_r, on='gym_id', how='inner')`: Esta línea combina los dos DataFrames utilizando una operación de fusión interna (inner join). Los pasos involucrados son:

  1. **Especificación de las Tablas a Combinar**: `df_final_product.merge(df_fitpass_r, ...)`: Esta parte del código indica que quieres fusionar el DataFrame `df_final_product` con el DataFrame `df_fitpass_r`.

  2. **Criterio de Fusión**: `on='gym_id'`: Aquí se especifica que la fusión debe realizarse en base a la columna `'gym_id'` que debe estar presente en ambos DataFrames. La operación de fusión alineará las filas de ambos DataFrames donde los valores de `'gym_id'` coinciden.

  3. **Tipo de Fusión**: `how='inner'`: Se utiliza una fusión interna, lo que significa que el DataFrame resultante (`df_work`) solo incluirá las filas que tienen una coincidencia en `'gym_id'` en ambos DataFrames. Las filas en `df_final_product` que no tengan una correspondencia en `df_fitpass_r`, y viceversa, serán excluidas del DataFrame resultante.

- `df_work`: Al final, el resultado de la fusión se almacena en el nuevo DataFrame `df_work`. Este DataFrame contendrá todas las columnas de ambos `df_final_product` y `df_fitpass_r`, pero solo las filas donde el `'gym_id'` coincide en ambos.

Esta operación es común en el análisis de datos, especialmente cuando se necesitan combinar diferentes conjuntos de datos en base a una clave común para obtener una vista más completa o realizar análisis más detallados. En tu caso, podría ser útil para combinar información detallada sobre gimnasios o estudios de fitness con otras métricas o datos relevantes para el análisis.

In [13]:
def generate_sql_query(lat, lon, activities):
    """
    Genera una consulta SQL basada en la latitud, longitud y actividades proporcionadas.

    :param lat: Latitud del usuario.
    :param lon: Longitud del usuario.
    :param activities: Lista de actividades de interés.
    :return: String con la consulta SQL.
    """

    # Inicia la consulta SQL base
    query = "SELECT * FROM cdmx_studios"

    # Añade condiciones basadas en la ubicación y actividades
    conditions = []

    if lat is not None and lon is not None:
        # Añade una condición basada en la proximidad (esto es un ejemplo y puede requerir ajuste)
        # Por ejemplo, puedes calcular la distancia entre las coordenadas del estudio y la ubicación del usuario
        # Aquí se asume que tienes una función 'calculate_distance' que puede hacer este cálculo
        conditions.append(f"calculate_distance(latitude, longitude, {lat}, {lon}) < cierto_umbral")

    if activities:
        # Añade condiciones para filtrar por actividades (asumiendo que las actividades son columnas en tu tabla)
        activity_conditions = [f"{activity} = 1" for activity in activities]
        conditions.append(" OR ".join(activity_conditions))

    if conditions:
        query += " WHERE " + " AND ".join(conditions)

    return query


In [7]:
import dash
import dash_leaflet as dl
from dash import dash_table, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import requests
import sqlalchemy as sa
import math

# Configuración de la conexión a la base de datos
db_name = 'fitpass'
db_user = 'postgres'
db_host = 'localhost'
db_password = 'skalas-puts-me-an-aplus-in-this-class'
database_url = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
engine = sa.create_engine(database_url)

# Inicialización de la aplicación Dash
app = dash.Dash(__name__)

# Layout de la Aplicación
app.layout = html.Div([
    html.H1("Dashboard de Estudios de Fitness"),
    dcc.Input(id='input-lat', type='text', placeholder='Latitud'),
    dcc.Input(id='input-lon', type='text', placeholder='Longitud'),
    dcc.Dropdown(
        id='activity-dropdown',
        options=[{'label': activity, 'value': activity} for activity in ['barre', 'box', 'crossfit', 'cycling', 'dance', 'ems', 'functional', 'gym', 'hiit', 'mma', 'pilates', 'pool', 'running', 'sports', 'virtual_class', 'wellness', 'yoga']],
        multi=True,
        placeholder="Selecciona actividades que te gustan"
    ),
    dcc.Dropdown(
        id='input-dislike',
        options=[{'label': activity, 'value': activity} for activity in ['barre', 'box', 'crossfit', 'cycling', 'dance', 'ems', 'functional', 'gym', 'hiit', 'mma', 'pilates', 'pool', 'running', 'sports', 'virtual_class', 'wellness', 'yoga']],
        multi=True,
        placeholder="Selecciona actividades que no te gustan"
    ),
    dcc.Input(id='input-frequency', type='number', placeholder='Número de clases al mes'),
     dcc.Dropdown(
        id='distance-dropdown',
        options=[
            {'label': 'Cerca', 'value': 'cerca'},
            {'label': 'Medio', 'value': 'medio'},
            {'label': 'Lejano', 'value': 'lejano'}
        ],
        placeholder="Selecciona la distancia"
    ),
    html.Button('Buscar', id='search-button', n_clicks=0),
    dl.Map(id='map-view', style={'width': '1000px', 'height': '500px'}, center=[19.432608, -99.133209], zoom=10, children=[
    dl.TileLayer()
    ]),
    dcc.Graph(id='fig_graph'),
    html.Div(id='table-view')  # Div para mostrar la tabla de resultados
])

# Función para calcular la distancia utilizando la fórmula de Haversine
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en kilómetros
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Función para generar la consulta SQL con las nuevas entradas
def generate_sql_query(lat, lon, love_activities, dislike_activities, frequency, distance_option):
    query = "SELECT * FROM cdmx_studios WHERE TRUE"
   
    if frequency:
        query += f" AND (max_classes >= {frequency})"

    return query


# Función para combinar DataFrames
def combine_dataframes(df1, df2):
    combined_df = pd.concat([df1, df2], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=['gym_id'])
    return combined_df

@app.callback(
    [Output('map-view', 'children'), Output('fig_graph', 'figure'), Output('table-view', 'children')],
    [Input('search-button', 'n_clicks')],
    [State('input-lat', 'value'), State('input-lon', 'value'), 
     State('activity-dropdown', 'value'), State('distance-dropdown', 'value'),
     State('input-dislike', 'value'), State('input-frequency', 'value')]
)
def update_outputs(n_clicks, lat, lon, activities, distance_option, dislikes, frequency):
    try:
        lat = float(lat) if lat else None
        lon = float(lon) if lon else None
    except ValueError:
        lat, lon = None, None

    df_combined = pd.DataFrame()  # Inicializa df_combined como un DataFrame vacío
    markers = [dl.TileLayer()]  # Inicializa markers con solo TileLayer

    if lat is not None and lon is not None:
        query = generate_sql_query(lat, lon, activities, dislikes, frequency, distance_option)
        df_studios = pd.read_sql_query(query, engine)
   # Calcula la distancia y filtra los estudios si se especificó la opción de distancia
        if distance_option:
            distance_limits = {'cerca': 5, 'medio': 15, 'lejano': 30}
            max_distance = distance_limits.get(distance_option, 10)  # Valor por defecto si la opción no se reconoce

            df_studios['distance'] = df_studios.apply(
                lambda row: calculate_distance(lat, lon, row['latitude'], row['longitude']), axis=1
            )
            df_studios = df_studios[df_studios['distance'] <= max_distance]
    if lat is not None and lon is not None:
        # Obtener estudios recomendados de la base de datos
        query = generate_sql_query(lat, lon, activities, dislikes, frequency, distance_option)
        df_studios = pd.read_sql_query(query, engine)
        
        if n_clicks > 0 and activities:
            # Hacer la solicitud a la API
            sample_request = {
                "location": {"latitude": lat, "longitude": lon}, 
                "love_activities": activities, 
                "hate_activities": dislikes, 
                "num_classes_per_month": frequency
            }
            response = requests.post('http://localhost:8080/predict', json=sample_request)
            response_json = response.json()
            if isinstance(response_json, list):
                df_recommendations = pd.DataFrame(response_json)
            elif isinstance(response_json, dict):
                df_recommendations = pd.DataFrame([response_json])
            else:
                raise ValueError("Respuesta JSON no reconocida")

            df_combined = combine_dataframes(df_studios, df_recommendations)

            if not df_combined.empty:
            # Crea marcadores para cada estudio en df_combined
                for _, row in df_combined.iterrows():
                    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
                        marker = dl.Marker(position=[row['latitude'], row['longitude']], children=[
                            dl.Tooltip(row['gym_name'])
                        ])
                        markers.append(marker)  
            # Modificar los valores de pro_status
            df_combined['pro_status'] = df_combined['pro_status'].apply(lambda x: 1 if x == 1 else -1)
    # Crear la gráfica de barras
    if not df_combined.empty:
        fig_graph = px.bar(df_combined, x='gym_name', y='pro_status',
                           title='Tipo de Estudio y Fitpass Pro Status',
                           labels={'pro_status': 'Estado Pro'})
    else:
        fig_graph = {}

    # Crear la tabla
        table = dash_table.DataTable(
                data=df_combined.to_dict('records'),
                columns=[{"name": i, "id": i} for i in df_combined.columns],
                style_table={'overflowX': 'scroll'}
            )

        # Asegúrate de retornar marcadores para el mapa
        return markers, fig_graph, table

    # Retornar elementos vacíos si no hay datos válidos
    return [dl.TileLayer()], html.Div(), html.Div()

# Ejecución del servidor
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

---------------------------------------------------------------------------
UndefinedColumn                           Traceback (most recent call last)
File ~/anaconda3/lib/python3.11/site-packages/sqlalchemy/engine/base.py:1819, in Connection._execute_context(
    self=<sqlalchemy.engine.base.Connection object>,
    dialect=<sqlalchemy.dialects.postgresql.psycopg2.PGDialect_psycopg2 object>,
    constructor=<bound method DefaultExecutionContext._init_stat...ostgresql.psycopg2.PGExecutionContext_psycopg2'>>,
    statement='SELECT * FROM cdmx_studios WHERE TRUE AND (max_classes >= 4)',
    parameters={},
    execution_options=immutabledict({}),
    *args=('SELECT * FROM cdmx_studios WHERE TRUE AND (max_classes >= 4)', []),
    **kw={}
)
   1818     if not evt_handled:
-> 1819         self.dialect.do_execute(
        self = <sqlalchemy.engine.base.Connection object at 0x15d6a2810>
        self.dialect = <sqlalchemy.dialects.postgresql.psycopg2.PGDialect_psycopg2 object at 0x15d78e250>
  

In [9]:
import dash
import dash_leaflet as dl
from dash import dash_table, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import pandas as pd
import requests
import sqlalchemy as sa
import math

# Configuración de la conexión a la base de datos
db_name = 'fitpass'
db_user = 'postgres'
db_host = 'localhost'
db_password = 'skalas-puts-me-an-aplus-in-this-class'
database_url = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
engine = sa.create_engine(database_url)

# Inicialización de la aplicación Dash
app = dash.Dash(__name__)

# Layout de la Aplicación
app.layout = html.Div([
    html.H1("Dashboard de Estudios de Fitness"),
    dcc.Input(id='input-lat', type='text', placeholder='Latitud'),
    dcc.Input(id='input-lon', type='text', placeholder='Longitud'),
    dcc.Dropdown(
        id='activity-dropdown',
        options=[{'label': activity, 'value': activity} for activity in ['barre', 'box', 'crossfit', 'cycling', 'dance', 'ems', 'functional', 'gym', 'hiit', 'mma', 'pilates', 'pool', 'running', 'sports', 'virtual_class', 'wellness', 'yoga']],
        multi=True,
        placeholder="Selecciona actividades que te gustan"
    ),
    dcc.Dropdown(
        id='input-dislike',
        options=[{'label': activity, 'value': activity} for activity in ['barre', 'box', 'crossfit', 'cycling', 'dance', 'ems', 'functional', 'gym', 'hiit', 'mma', 'pilates', 'pool', 'running', 'sports', 'virtual_class', 'wellness', 'yoga']],
        multi=True,
        placeholder="Selecciona actividades que no te gustan"
    ),
    dcc.Input(id='input-frequency', type='number', placeholder='Número de clases al mes'),
    dcc.Dropdown(
        id='distance-dropdown',
        options=[
            {'label': 'Cerca', 'value': 'cerca'},
            {'label': 'Medio', 'value': 'medio'},
            {'label': 'Lejano', 'value': 'lejano'}
        ],
        placeholder="Selecciona la distancia"
    ),
    html.Button('Buscar', id='search-button', n_clicks=0),
    dl.Map(id='map-view', style={'width': '1000px', 'height': '500px'}, center=[19.432608, -99.133209], zoom=10, children=[
        dl.TileLayer()
    ]),
    dcc.Graph(id='fig_graph'),
    html.Div(id='table-view')  # Div para mostrar la tabla de resultados
])

# Función para calcular la distancia utilizando la fórmula de Haversine
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en kilómetros
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

@app.callback(
    [
        Output('map-view', 'children'),  # Para los marcadores del mapa
        Output('fig_graph', 'figure'),  # Para la gráfica
        Output('table-view', 'children')  # Para la tabla
    ],
    [
        Input('search-button', 'n_clicks')
    ],
    [
        State('input-lat', 'value'), State('input-lon', 'value'), 
        State('activity-dropdown', 'value'), State('distance-dropdown', 'value'),
        State('input-dislike', 'value'), State('input-frequency', 'value')
    ]
)
def update_outputs(n_clicks, lat, lon, activities, distance_option, dislikes, frequency):
    # Validar y convertir las entradas
    try:
        lat = float(lat) if lat else None
        lon = float(lon) if lon else None
    except ValueError:
        lat, lon = None, None
    # Inicializar componentes vacíos
    map_markers = [dl.TileLayer()]
    graph_figure = {}
    table_data = html.Div()
    if n_clicks > 0 and activities:
        # Preparar la petición a la API
        sample_request = {
            "location": {"latitude": lat, "longitude": lon}, 
            "preferences":{"love_activities": activities, 
            "hate_activities": dislikes},
            "is_pro": 1,
            "max_allowed_classes_per_class": 4,
            "num_classes_per_month": frequency
        }
        response = requests.post('http://localhost:8080/predict', json=sample_request)
        if response.status_code == 200:
            response_json = response.json()
            df_studios = pd.DataFrame(response_json)

            # Actualizar el Mapa
            map_markers = [dl.TileLayer()]
            for _, row in df_studios.iterrows():
                marker = dl.Marker(position=[row['latitude'], row['longitude']], children=[
                    dl.Tooltip(row['gym_name'])
                ])
                map_markers.append(marker)

            # Actualizar la Gráfica
            graph_figure = px.bar(df_studios, x='gym_name', y='some_metric', 
                                  title='Métrica por Gimnasio', labels={'some_metric': 'Métrica'})

            # Actualizar la Tabla
            table_data = dash_table.DataTable(
                data=df_studios.to_dict('records'),
                columns=[{"name": i, "id": i} for i in df_studios.columns],
                style_table={'overflowX': 'scroll'}
            )
        else:
            # Manejar errores en la respuesta
            print("Error en la petición a la API")

    # Retornar los componentes actualizados
    return map_markers, graph_figure, table_data

# Ejecución del servidor
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


Error en la petición a la API
